# Documentation
Author:
Randi Ang

This set of code seeks to calculate the distance covered between to GPS coordinates, taken at fixed time intervals.
Depending on the interval time, the distance covered may be your velocity if its unit of measure matches your use case. E.g. if the data collected is at 1 second intervals, and if the distance covered in 1s is 5m, that would translate to a speed of 5m/s; but if your time interval is different say every 0.5s, and the distance covered in 0.5 seconds is 5m,that would translate to a speed of 10m/s.

Radius of the Earth
Equatorial radius (km)	        6378.137    
Polar radius (km)               6356.752         
Volumetric mean radius (km)     6371.000

Refeerence:
https://stackoverflow.com/questions/55909305/using-geopy-in-a-dataframe-to-get-distances
https://geopy.readthedocs.io/en/stable/#module-geopy.distance
https://www.researchgate.net/post/What_is_the_best_formula_in_finding_the_distances_between_one_coordinate_in_earth_to_another_given_the_longitude_and_latitude
https://nssdc.gsfc.nasa.gov/planetary/factsheet/earthfact.html

In [1]:
# Pandas package is used for general data processing
import pandas as pd

# Create dataframe with some sample data as a proof of concept
df = pd.DataFrame(
    [
        (-25.145439,  -54.294871),
        (-24.144564,  -54.240094),
        (-24.142564,  -54.198901),
        (-24.140093,  52.119021),
    ],
    columns=['latitude', 'longitude']
)

# GeoPy package is used to calculate distance
# When calculating distance, one needs to factor in the spherical radius of the celestial object
# where the measurements are taken; in this case, GeoPy takes this into account by default
from geopy import Point
from geopy import distance


# Combine coordinates as a single GPS point in a new column
df['point'] = df.apply(lambda row: Point(latitude=row['latitude'], longitude=row['longitude']), axis=1)

In [2]:
# Check output
df.head()

,latitude,longitude,point
0,-25.145439,-54.294871,"25 8m 43.5804s S, 54 17m 41.5356s W"
1,-24.144564,-54.240094,"24 8m 40.4304s S, 54 14m 24.3384s W"
2,-24.142564,-54.198901,"24 8m 33.2304s S, 54 11m 56.0436s W"
3,-24.140093,52.119021,"24 8m 24.3348s S, 52 7m 8.4756s E"


In [3]:
# Get the previous data point at x intervals, in this case shift by 1
# If the previous value is not available, treat it as "None"
df['point_next'] = df['point'].shift(1) 
df.loc[df['point_next'].isna(), 'point_next'] = None

In [4]:
# Check output
df.head()

,latitude,longitude,point,point_next
0,-25.145439,-54.294871,"25 8m 43.5804s S, 54 17m 41.5356s W",None
1,-24.144564,-54.240094,"24 8m 40.4304s S, 54 14m 24.3384s W","25 8m 43.5804s S, 54 17m 41.5356s W"
2,-24.142564,-54.198901,"24 8m 33.2304s S, 54 11m 56.0436s W","24 8m 40.4304s S, 54 14m 24.3384s W"
3,-24.140093,52.119021,"24 8m 24.3348s S, 52 7m 8.4756s E","24 8m 33.2304s S, 54 11m 56.0436s W"


In [5]:
# Calculate distance in m if 2 GPS coordinates are available
# Default calculation using geodesic method
# WGS-84 is the default ellipsoid method for geodesic models
# WGS 84 is the reference coordinate system used by the GPS
# It is currently the main standard used for cartography, GIS and navigation
# Recommended method
df['distance_m_default'] = df.apply(lambda row: distance.distance(row['point'], row['point_next']).m if row['point_next'] is not None else float('nan'), axis=1)
df.head()

,latitude,longitude,point,point_next,distance_m_default
0,-25.145439,-54.294871,"25 8m 43.5804s S, 54 17m 41.5356s W",None,NaN
1,-24.144564,-54.240094,"24 8m 40.4304s S, 54 14m 24.3384s W","25 8m 43.5804s S, 54 17m 41.5356s W",1.110032e+05
2,-24.142564,-54.198901,"24 8m 33.2304s S, 54 11m 56.0436s W","24 8m 40.4304s S, 54 14m 24.3384s W",4.192654e+03
3,-24.140093,52.119021,"24 8m 24.3348s S, 52 7m 8.4756s E","24 8m 33.2304s S, 54 11m 56.0436s W",1.044966e+07


In [6]:
# Calculate distance in km if 2 GPS coordinates are available
# Default calculation using geodesic method
# WGS-84 is the default ellipsoid method for geodesic models
# WGS 84 is the reference coordinate system used by the GPS
# It is currently the main standard used for cartography, GIS and navigation
# Recommended method
df['distance_km_default'] = df.apply(lambda row: distance.distance(row['point'], row['point_next']).km if row['point_next'] is not None else float('nan'), axis=1)
df

,latitude,longitude,point,point_next,distance_m_default,distance_km_default
0,-25.145439,-54.294871,"25 8m 43.5804s S, 54 17m 41.5356s W",None,NaN,NaN
1,-24.144564,-54.240094,"24 8m 40.4304s S, 54 14m 24.3384s W","25 8m 43.5804s S, 54 17m 41.5356s W",1.110032e+05,111.003172
2,-24.142564,-54.198901,"24 8m 33.2304s S, 54 11m 56.0436s W","24 8m 40.4304s S, 54 14m 24.3384s W",4.192654e+03,4.192654
3,-24.140093,52.119021,"24 8m 24.3348s S, 52 7m 8.4756s E","24 8m 33.2304s S, 54 11m 56.0436s W",1.044966e+07,10449.661388


In [7]:
# Calculate distance in km if 2 GPS coordinates are available
# Calculation using geodesic method
# WGS-84 is the default ellipsoid method for geodesic models
# WGS 84 is the reference coordinate system used by the GPS
# It is currently the main standard used for cartography, GIS and navigation
# Recommended method
df['distance_km_geodesic'] = df.apply(lambda row: distance.geodesic(row['point'], row['point_next']).km if row['point_next'] is not None else float('nan'), axis=1)
df.head()

,latitude,longitude,point,point_next,distance_m_default,distance_km_default,distance_km_geodesic
0,-25.145439,-54.294871,"25 8m 43.5804s S, 54 17m 41.5356s W",None,NaN,NaN,NaN
1,-24.144564,-54.240094,"24 8m 40.4304s S, 54 14m 24.3384s W","25 8m 43.5804s S, 54 17m 41.5356s W",1.110032e+05,111.003172,111.003172
2,-24.142564,-54.198901,"24 8m 33.2304s S, 54 11m 56.0436s W","24 8m 40.4304s S, 54 14m 24.3384s W",4.192654e+03,4.192654,4.192654
3,-24.140093,52.119021,"24 8m 24.3348s S, 52 7m 8.4756s E","24 8m 33.2304s S, 54 11m 56.0436s W",1.044966e+07,10449.661388,10449.661388


In [8]:
# Calculate distance in km if 2 GPS coordinates are available
# Calculation using Great_Circle method
# The Earth's radius is used as a default
# Assumes that the Earth is a perfect sphere, but the Earth (and most other Celestial Entitites) are Ellipsoids
df['distance_km_greatCircle'] = df.apply(lambda row: distance.great_circle(row['point'], row['point_next']).km if row['point_next'] is not None else float('nan'), axis=1)
df.head()



,latitude,longitude,point,point_next,distance_m_default,distance_km_default,distance_km_geodesic,distance_km_greatCircle
0,-25.145439,-54.294871,"25 8m 43.5804s S, 54 17m 41.5356s W",None,NaN,NaN,NaN,NaN
1,-24.144564,-54.240094,"24 8m 40.4304s S, 54 14m 24.3384s W","25 8m 43.5804s S, 54 17m 41.5356s W",1.110032e+05,111.003172,111.003172,111.429982
2,-24.142564,-54.198901,"24 8m 33.2304s S, 54 11m 56.0436s W","24 8m 40.4304s S, 54 14m 24.3384s W",4.192654e+03,4.192654,4.192654,4.185688
3,-24.140093,52.119021,"24 8m 24.3348s S, 52 7m 8.4756s E","24 8m 33.2304s S, 54 11m 56.0436s W",1.044966e+07,10449.661388,10449.661388,10432.801875


In [9]:
# Other Ellipsoids models that one may use
distance.ELLIPSOIDS

{'WGS-84': (6378.137, 6356.7523142, 0.0033528106647474805),
 'GRS-80': (6378.137, 6356.7523141, 0.003352810681182319),
 'Airy (1830)': (6377.563396, 6356.256909, 0.0033408506414970775),
 'Intl 1924': (6378.388, 6356.911946, 0.003367003367003367),
 'Clarke (1880)': (6378.249145, 6356.51486955, 0.003407561378699334),
 'GRS-67': (6378.16, 6356.774719, 0.003352891869237217)}

In [10]:
# Calculate distance in km if 2 GPS coordinates are available
# Calculation using geodesic method
# GRS-80 is used in this example
# Choose other Ellipsoid models where need be, by substituting the ellipsoid argument value with the options listed above
df['distance_km_GRS80'] = df.apply(lambda row: distance.geodesic(row['point'], row['point_next'], ellipsoid='GRS-80').km if row['point_next'] is not None else float('nan'), axis=1)
df.head()

,latitude,longitude,point,point_next,distance_m_default,distance_km_default,distance_km_geodesic,distance_km_greatCircle,distance_km_GRS80
0,-25.145439,-54.294871,"25 8m 43.5804s S, 54 17m 41.5356s W",None,NaN,NaN,NaN,NaN,NaN
1,-24.144564,-54.240094,"24 8m 40.4304s S, 54 14m 24.3384s W","25 8m 43.5804s S, 54 17m 41.5356s W",1.110032e+05,111.003172,111.003172,111.429982,111.003172
2,-24.142564,-54.198901,"24 8m 33.2304s S, 54 11m 56.0436s W","24 8m 40.4304s S, 54 14m 24.3384s W",4.192654e+03,4.192654,4.192654,4.185688,4.192654
3,-24.140093,52.119021,"24 8m 24.3348s S, 52 7m 8.4756s E","24 8m 33.2304s S, 54 11m 56.0436s W",1.044966e+07,10449.661388,10449.661388,10432.801875,10449.661388


In [11]:
# Drop column for "point_next" when done
df = df.drop('point_next', axis=1)
df.head()

,latitude,longitude,point,distance_m_default,distance_km_default,distance_km_geodesic,distance_km_greatCircle,distance_km_GRS80
0,-25.145439,-54.294871,"25 8m 43.5804s S, 54 17m 41.5356s W",NaN,NaN,NaN,NaN,NaN
1,-24.144564,-54.240094,"24 8m 40.4304s S, 54 14m 24.3384s W",1.110032e+05,111.003172,111.003172,111.429982,111.003172
2,-24.142564,-54.198901,"24 8m 33.2304s S, 54 11m 56.0436s W",4.192654e+03,4.192654,4.192654,4.185688,4.192654
3,-24.140093,52.119021,"24 8m 24.3348s S, 52 7m 8.4756s E",1.044966e+07,10449.661388,10449.661388,10432.801875,10449.661388


# End